In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics


In [4]:
#tratamento de dados
#transformar as 4 colunas com string em numerico
def cleanData(data) :
    for col in data.columns.values:
        data[col] = data[col].astype('string')
    #----------
    for col in data.columns.values:
        data[col] = data[col].astype('float',errors = 'ignore')
    #-----------
    data['Gender']=data.Gender.map({'Male': 1, 'Female': 2})
    data['Dyslexia']=data.Dyslexia.map({'No': 0, 'Yes': 1})
    data['Nativelang']=data.Nativelang.map({'No': 0, 'Yes': 1})
    data['Otherlang']=data.Otherlang.map({'No': 0, 'Yes': 1})

In [5]:
desktopData= pd.read_csv("/content/Dyslexia_dataset.csv", sep = ';', encoding = 'utf-8')
cleanData(desktopData)

In [7]:
commonalityColumns = ['Gender','Nativelang','Otherlang','Age' , 'Dyslexia']
for i in  range(30):
    if((i>=0 and i<12) or (i>=13 and i<17) or i==21 or i==22 or i==29):
        commonalityColumns.append('Clicks'+str(i+1))
        commonalityColumns.append('Hits'+str(i+1))
        commonalityColumns.append('Misses'+str(i+1))
        commonalityColumns.append('Score'+str(i+1))
        commonalityColumns.append('Accuracy'+str(i+1))
        commonalityColumns.append('Missrate'+str(i+1))

desktopData=desktopData.loc[:,commonalityColumns]

In [9]:
y=desktopData['Dyslexia']
X=desktopData.loc[:, desktopData.columns != 'Dyslexia']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42)# Bilancia il dataset
X_train_sm, y_train_sm = SMOTE(random_state=42).fit_resample(X_train, y_train)
print("Class distribution after SMOTE:", y_train_sm.value_counts())

Class distribution after SMOTE: Dyslexia
0    2603
1    2603
Name: count, dtype: int64


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix # Import classification_report and confusion_matrix
from imblearn.ensemble import BalancedRandomForestClassifier

weights = {0: 1, 1: 6}  # Dai più peso alla classe 1 rispetto alla classe 0
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    class_weight=weights,  # handle class imbalance
    random_state=42
)

rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)

print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       649
           1       0.62      0.39      0.48        80

    accuracy                           0.91       729
   macro avg       0.77      0.68      0.71       729
weighted avg       0.89      0.91      0.90       729

Confusion Matrix:
[[630  19]
 [ 49  31]]
